In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import html

In [2]:
df = pd.read_csv('../data_collecting/raw_data.csv')
df

,tweetId,author_id,tweet,lang,created_at
0,1490905225930190849,2578185314,"In 2021, Reliance acquired @faradion_uk Ltd, l...",en,2022-02-08 04:28:00+00:00
1,1490905219286245376,291609629,"Hong Kong to tighten social-distancing rules, ...",en,2022-02-08 04:27:58+00:00
2,1490905197375553537,1320755164827938816,"you would know if someone lie, why those peopl...",en,2022-02-08 04:27:53+00:00
3,1490905197123620864,983016607,Doing. Experience and environment may shape yo...,en,2022-02-08 04:27:53+00:00
4,1490905176953454597,2675645600,@jenhada @airallianceHOU How does the communit...,en,2022-02-08 04:27:48+00:00
...,...,...,...,...,...
10045,1495106833614262274,995443722,Staff in climate sceptic group recruited by @C...,en,2022-02-19 18:43:41+00:00
10046,1495106831496196096,1353769293914247168,"NEMA , for the better environment I advise you...",en,2022-02-19 18:43:41+00:00
10047,1495106818959360013,79232783,@BruceAllmiighty Because society allows it and...,en,2022-02-19 18:43:38+00:00
10048,1495106813196333058,1602205790,"Due to his long list of offenses, wildlife off...",en,2022-02-19 18:43:36+00:00


In [4]:
# check is all the data in english
tweet_by_lang = df.groupby("lang")["tweet"].count()
print(tweet_by_lang)

lang
en    10000
Name: tweet, dtype: int64


In [5]:
import numpy as np
import re

# need to use a stopword library
stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]


def clean_tweet(tweet):
    temp = tweet.lower()
    temp = temp.replace('\n', ' ')
    temp = html.unescape(temp)
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)  # removing handle 
    temp = re.sub("#[A-Za-z0-9_]+","", temp)  # removing hastags
    temp = re.sub(r'http\S+', '', temp) # removing any link
    temp = re.sub('[()!?]', ' ', temp)  # remove the punct 
    temp = re.sub('\[.*?\]',' ', temp)  # remove any weird symbol
    temp = re.sub("[^a-z0-9]"," ", temp) # remove any numbers
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords] 
    temp = " ".join(word for word in temp)
    return temp

In [6]:
results = [clean_tweet(tw) for tw in df['tweet']]

In [7]:
df['clean_tweet'] = results

In [8]:
df.head(10)

,tweetId,author_id,tweet,lang,created_at,clean_tweet
0,1490905225930190849,2578185314,"In 2021, Reliance acquired @faradion_uk Ltd, l...",en,2022-02-08 04:28:00+00:00,2021 reliance acquired ltd leading global sodi...
1,1490905219286245376,291609629,"Hong Kong to tighten social-distancing rules, ...",en,2022-02-08 04:27:58+00:00,hong kong tighten social distancing rules stan...
2,1490905197375553537,1320755164827938816,"you would know if someone lie, why those peopl...",en,2022-02-08 04:27:53+00:00,you would know if someone lie why those people...
3,1490905197123620864,983016607,Doing. Experience and environment may shape yo...,en,2022-02-08 04:27:53+00:00,doing experience environment may shape you nam...
4,1490905176953454597,2675645600,@jenhada @airallianceHOU How does the communit...,en,2022-02-08 04:27:48+00:00,how does community hold valero accountable tox...
5,1490905163321987073,829956505851998209,@AUThackeray is noise pollution not harmful fo...,en,2022-02-08 04:27:45+00:00,is noise pollution not harmful environment
6,1490905158313967618,402452107,Putting a drink in a fridge/freezer “cools” it...,en,2022-02-08 04:27:44+00:00,putting drink fridge freezer cools it down ess...
7,1490905153817309185,1400023746241433601,@panda_airdrop24 @kakanftworld What makes you ...,en,2022-02-08 04:27:43+00:00,what makes you feel confident about survival s...
8,1490905153016377344,2536508238,Circumstances is whatever might be this: be wa...,en,2022-02-08 04:27:42+00:00,circumstances is whatever might be this be war...
9,1490905142048489476,1374004072898957316,@CryptoLegacyEN @NFT11_Official Most P2E games...,en,2022-02-08 04:27:40+00:00,most p2e games start giving away lot tokens fi...


In [9]:
df.to_csv('clean_data.csv', index=False)